In [0]:
%run ./00_config

In [0]:
# --- 顧客マスタの更新 (MVの「最新状態反映」を検証) ---
# U04: 新規ユーザー 伊藤さん (メールアドレスに @ がない不正形式)
users = """user_id,name,age,gender,email,region,registration_date,email_opt_in,last_updated,operation
U04,伊藤 三郎,35,Male,ito_invalid_email,愛知,2025-01-04,true,2025-01-04 12:00:00,INSERT
"""

# --- 不正な購買履歴 (Expectations検証用) ---
# 東エリアから届く初期データ
# 106: 金額 0 (期待値：amount > 0 -> 警告(WARN))
# 107: ユーザーID欠損 (期待値：user_id IS NOT NULL -> 停止(FAIL))
transactions_east = """order_id,user_id,product_id,amount,quantity,transaction_date
106,U01,P01,0,1,2025-01-04 13:00:00
107,,P02,5000,2,2025-01-04 13:00:00
"""

# 西エリアから届く初期データ
# 108: 1900年/異常な過去日付 (期待値：order_date >= '2020-01-01' -> 除外(DROP))
transactions_west = """order_id,user_id,product_id,amount,quantity,transaction_date
108,U03,P03,35000,1,1900-01-01 13:00:00
"""

In [0]:
import random
import string

def generate_unique_code(length=5):
    chars = string.ascii_lowercase + string.digits
    return ''.join(random.sample(chars, length))

random_suffix = generate_unique_code()


# 顧客マスタの書き出し (CSV)
dbutils.fs.put(f"{users_dir}users_{random_suffix}.csv", users, True)
print(f"{users_dir}users_{random_suffix}.csv")

# 購入履歴の書き出し (CSV)
dbutils.fs.put(f"{transactions_east_dir}transactions_east_{random_suffix}.csv", transactions_east, True)
print(f"{transactions_east_dir}transactions_east_{random_suffix}.csv")
dbutils.fs.put(f"{transactions_west_dir}transactions_west_{random_suffix}.csv", transactions_west, True)
print(f"{transactions_west_dir}transactions_west_{random_suffix}.csv")

print("")
print("✅ 03_不正データ追加：完了しました。")